In [142]:
from copy import deepcopy
import numpy as np
import random
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, transpile, assemble
from qiskit.providers import backend
from qiskit.aqua.components.optimizers import COBYLA

In [185]:
UP = "00"
DOWN = "01"
LEFT = "10"
RIGHT = "11"
ACTIONS = [UP, DOWN, LEFT, RIGHT]
# Define a matrix
gridSize = [3,3]
matrix = np.empty((3, 3),dtype=object)
def genmatrix(shape):
    for i in range(0,shape[0]):
        for j in range(0,shape[1]):
            matrix[i][j] = ACTIONS[random.randint(0,3)]
    return matrix
matrix = genmatrix(gridSize)
print(matrix)

[['11' '00' '00']
 ['10' '10' '10']
 ['01' '00' '10']]


In [144]:
target = genmatrix(gridSize)
print(target)

[['00' '01' '00']
 ['11' '01' '00']
 ['00' '10' '01']]


In [145]:
def circuits_check(matrix,target):
    cool = 0 
    good = 1
    normal = 2
    
    if matrix.shape != target.shape: 
        raise ValueError(f"Different shapes error")
    else:
        result = np.empty((3, 3),dtype=int)
        for i in range(0,matrix.shape[0]):
            for j in range(0,matrix.shape[1]):
                
                if matrix[i][j] == target[i][j]:
                    result[i][j] = cool
                    
                elif ((matrix[i][j] == UP and target[i][j] == DOWN)\
                    or (matrix[i][j] == DOWN and target[i][j] == UP) \
                    or (matrix[i][j] == LEFT and target[i][j] == RIGHT) \
                    or (matrix[i][j] == RIGHT and target[i][j] == LEFT)):
                    result[i][j] = normal
                    
                elif (matrix[i][j] != target[i][j]):
                    result[i][j] = good
                    
    return result

In [146]:
print(circuits_check(matrix,target))

[[0 0 0]
 [0 0 0]
 [0 0 0]]


In [147]:
# Define a dictionary to work with:
d = {(0, 0): (np.array([0., 0., 0., 0., 0., 0.]), 0.0, 0),
     (0, 1): (np.array([0., 0., 0., 0., 0., 0.]), -1.0, 37), 
     (0, 2): (np.array([0., 0., 0., 0., 0., 0.]), 0.0, 0), 
     (1, 0): (np.array([0., 0., 0., 0., 0., 0.]), 0.0, 37),
     (1, 1): (np.array([0., 0., 0., 0., 0., 0.]), -1.0, 37),
     (1, 2): (np.array([0., 0., 0., 0., 0., 0.]), -1.0, 37),
     (2, 0): (np.array([-0.02332911,  1.45054987,  1.4877359 ,  1.27548996,  1.48778515, 1.27549177]), 0.0, 37),
     (2, 1): (np.array([0., 0., 0., 0., 0., 0.]), -1.0, 37), 
     (2, 2): (np.array([0., 0., 0., 0., 0., 0.]), -1.0, 37)}

In [181]:
qt = {(0, 0): np.array([0., 0., 0., 0.]), (0, 1): np.array([0., 0., 0., 0.]), (0, 2): np.array([0., 0., 0., 0.]), (1, 0): np.array([100.,   0.,   0.,   0.]), (1, 1): np.array([0., 0., 0., 0.]), (1, 2): np.array([0., 0., 0., 0.]), (2, 0): np.array([97.        , 89.20848412, 94.06      ,  0.        ]), (2, 1): np.array([0., 0., 0., 0.]), (2, 2): np.array([0., 0., 0., 0.])}

In [148]:
for key, value in d.items() :
    print (key, value)

(0, 0) (array([0., 0., 0., 0., 0., 0.]), 0.0, 0)
(0, 1) (array([0., 0., 0., 0., 0., 0.]), -1.0, 37)
(0, 2) (array([0., 0., 0., 0., 0., 0.]), 0.0, 0)
(1, 0) (array([0., 0., 0., 0., 0., 0.]), 0.0, 37)
(1, 1) (array([0., 0., 0., 0., 0., 0.]), -1.0, 37)
(1, 2) (array([0., 0., 0., 0., 0., 0.]), -1.0, 37)
(2, 0) (array([-0.02332911,  1.45054987,  1.4877359 ,  1.27548996,  1.48778515,
        1.27549177]), 0.0, 37)
(2, 1) (array([0., 0., 0., 0., 0., 0.]), -1.0, 37)
(2, 2) (array([0., 0., 0., 0., 0., 0.]), -1.0, 37)


In [179]:
keys = list(d.keys())
print(keys[0][0])

0


In [183]:
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 1000 # number of measurements 
optimizer = COBYLA(maxiter=500, tol=0.0001) # off the shelf


def qcMaker(params):
    qr = QuantumRegister(2, name="q")
    cr = ClassicalRegister(2, name="c")
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.u3(params[3], params[4], params[5], qr[1])
    qc.measure(qr, cr)
    return qc

def calcActions(d):
 
    actions = dict()
    for key in d.keys():
        qc = qcMaker(d[key][0])
        t_qc = transpile(qc, backend)
        job = assemble(t_qc, shots=NUM_SHOTS)
        rlt = backend.run(job).result()
        counts = rlt.get_counts(qc) 
        action = max(counts, key = counts.get)
        actions[key] = action
        
    return actions

NumRows = 3
NumColumns = 3

def newPosition(position, action):
        p = position
        if action == UP:
            p[0] = max(0, p[0] - 1)
        elif action == DOWN:
            p[0] = min(NumRows - 1, p[0] + 1)
        elif action == LEFT:
            p[1] = max(0, p[1] - 1)
        elif action == RIGHT:
            p[1] = min(NumColumns - 1, p[1] + 1)
        else:
            raise ValueError(f"Unkown action")
        return p

    # qt is a dict with all q-values
def calcPerformance(d, actions: dict, qt: dict):
    
    matrix = np.empty((3, 3),dtype=object)
    targetmatrix = np.empty((3, 3),dtype=object)
    for key in d.keys():
        action = ACTIONS[int(actions[key],2)] # "00","01"
        matrix[key[0]][key[1]] = action
        targetAction = ACTIONS[np.argmax(qt[key])] # "00","01"
        targetmatrix[key[0]][key[1]] = targetAction
    return circuits_check(matrix,targetmatrix)

In [184]:
actions = calcPerformance(d,calcActions(d),qt)
print(actions)

[[0 0 0]
 [0 0 0]
 [0 0 0]]
